In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import json
import os

path = r'C:\Users\ruanv\OneDrive\Documentos\Mestrado\Projeto de Mestrado\bases'

df = pd.read_csv(os.path.join(path,'demandaTrafos.csv'), sep=';')
df.rename(columns={
    'Unnamed: 3': 'Valor_Pot'
}, inplace=True)

subestacoes = pd.read_excel(os.path.join(path,'subestacoes.xlsx'), usecols='A:G')

subestacoes['Equipamento Medição'] = subestacoes['SUBESTACAO']+'_DJ_'+subestacoes['DISJUNTOR']
subestacoes['TRAFO'] = subestacoes['SUBESTACAO']+"-"+subestacoes['TRAFO']
subestacoes = subestacoes[['TRAFO', 'Equipamento Medição', 'POTENCIA (MVA)']]

with open(os.path.join(path,'schema_trafos.json'), 'r', encoding='utf-8') as file:
    schema = json.load(file)

In [2]:
df['Potencia Ativa'] = df.apply(lambda row: row['Valor_Pot'] if row['Métrica'] == 'Potência Ativa' else '0', axis=1)
df['Potencia Reativa'] = df.apply(lambda row: row['Valor_Pot'] if row['Métrica'] == 'Potência Reativa' else '0', axis=1)

df = df.groupby(['Equipamento Medição', 'Data/Hora Medição'], as_index=False).agg(
    {'Potencia Ativa': 'first', 'Potencia Reativa': 'last'}
)

df = pd.merge(df, subestacoes, on='Equipamento Medição')
df = df.drop_duplicates()

df = df[['TRAFO', 'Data/Hora Medição', 'Potencia Ativa', 'Potencia Reativa', 'POTENCIA (MVA)']]

df['Potencia Reativa'] = df['Potencia Reativa'].str.replace(',', '.')
df['Potencia Ativa'] = df['Potencia Ativa'].str.replace(',', '.')

df['Potencia Reativa'] = pd.to_numeric(df['Potencia Reativa'], errors='coerce')
df['Potencia Ativa'] = pd.to_numeric(df['Potencia Ativa'], errors='coerce')


In [4]:
df.to_csv(r'C:\Users\ruanv\OneDrive\Documentos\Mestrado\Projeto de Mestrado\bases\demandaTrafos_2018-2023.csv', sep=';', encoding='latin-1', index=False)